In [100]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Make NumPy printouts easier to read.
np.set_printoptions(precision=5, suppress=True)

import tensorflow as tf
print(tf.__version__)

2.13.0


In [101]:
dataset = pd.read_csv('../dane_scalone.csv')

dataset

,Pora_roku,Dzien_tygodnia,Czas_dnia,Przeplyw
0,2.0,1.0,0.000000,0.0
1,2.0,1.0,0.000694,0.0
2,2.0,1.0,0.001389,0.0
3,2.0,1.0,0.002083,0.0
4,2.0,1.0,0.002778,0.0
...,...,...,...,...
1048570,4.0,1.0,0.173611,0.0
1048571,4.0,1.0,0.174306,0.0
1048572,4.0,1.0,0.175000,0.0
1048573,4.0,1.0,0.175694,0.0


In [102]:
dataset=dataset.drop(columns="Pora_roku")
x=dataset.drop(columns="Przeplyw")
y=dataset.drop(columns="Dzien_tygodnia")
y=y.drop(columns="Czas_dnia")

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [103]:
y_train

,Przeplyw
408561,0.0
70143,0.0
708782,0.0
572694,0.0
774181,0.0
...,...
259178,0.0
365838,0.0
131932,0.0
671155,0.0


In [104]:
model = tf.keras.models.Sequential([
      tf.keras.layers.Dense(256, activation='relu', input_shape=(2,)),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dense(64, activation='relu'),
      tf.keras.layers.Dense(32, activation='relu'),
      tf.keras.layers.Dense(1,activation='sigmoid') 
])

In [105]:
model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.01),
              loss="mse",
              metrics=['mae'])

In [106]:
model.fit(x_train, y_train, epochs=5, batch_size=64)

Epoch 1/5
13108/13108 [==============================] - 12s 877us/step - loss: 1.2744 - accuracy: 0.9853
Epoch 2/5
13108/13108 [==============================] - 11s 865us/step - loss: 1.2744 - accuracy: 0.9853
Epoch 3/5
13108/13108 [==============================] - 11s 857us/step - loss: 1.2744 - accuracy: 0.9853
Epoch 4/5
13108/13108 [==============================] - 11s 852us/step - loss: 1.2744 - accuracy: 0.9853
Epoch 5/5
13108/13108 [==============================] - 11s 853us/step - loss: 1.2744 - accuracy: 0.9853


In [107]:
model.evaluate(x_test,  y_test, verbose=2)

6554/6554 - 3s - loss: 1.2734 - accuracy: 0.9852 - 3s/epoch - 486us/step


[1.2734204530715942, 0.9851751327514648]

In [108]:
sample_time=2;      #[s]
symulation_time=60; #[min]

current_hours=12;   #[h]
current_minutes=00; #[min]
current_day=1;      #[label, pon-1, wt-2,...,nd-7]

In [109]:
sample_time_normalized=sample_time/(60*60*24);              #[min - normalized]
symulation_time_normlized=symulation_time/(24*60)           #[min - normalized]
current_time=(current_hours*60+current_minutes)/(24*60)     #[min - normalized]

endtime=current_time+symulation_time_normlized;
if endtime<1:
    flow_time=np.arange(current_time,endtime,sample_time_normalized)                    #[min - normalized]
    flow_time=np.vstack((current_day*np.ones_like(flow_time, dtype=int),flow_time)).T   #[label, min]
else:
    temp_1=np.arange(current_time,1,sample_time_normalized)                             #[min - normalized]
    temp_1=np.vstack((current_day*np.ones_like(temp_1, dtype=int),temp_1)).T            #[label, min]

    temp_2=np.arange(0,endtime-1,sample_time_normalized)                                #[min - normalized]
    temp_2=np.vstack(((current_day+1)*np.ones_like(temp_2, dtype=int),temp_2)).T        #[label, min] 

    flow_time=np.concatenate((temp_1,temp_2))                                           #[label, min]
flow_time

array([[1.     , 0.5    ],
       [1.     , 0.50002],
       [1.     , 0.50005],
       ...,
       [1.     , 0.5416 ],
       [1.     , 0.54162],
       [1.     , 0.54164]])

In [110]:
flow_result = model.predict(flow_time)                   # [l/min]
water_usage_predict=np.sum(flow_result)*(sample_time/60) # [l]

water_usage_predict

57/57 [==============================] - 0s 749us/step


-718.1513020833333

In [111]:
flow_result

array([[-11.89304],
       [-11.89312],
       [-11.89321],
       ...,
       [-12.04524],
       [-12.04533],
       [-12.04541]], dtype=float32)